In [41]:
import requests

def get_companies_by_nace_code_and_city(nace_code, city, max_results=5000):
    # Initialiser variabler for paginering
    base_url = 'https://data.brreg.no/enhetsregisteret/api/enheter'
    page = 0
    all_companies = []
    page_size = 100  # API-et har ofte en maksgrense på 100 per side
    total_fetched = 0

    while total_fetched < max_results:
        # Sett opp URL med næringskode, poststed (by) og paginering
        url = f'{base_url}?naeringskode={nace_code}&forretningsadresse.poststed={city}&page={page}&size={page_size}'
        
        # Send forespørselen til API-et
        response = requests.get(url)

        # Sjekk om forespørselen var vellykket
        if response.status_code == 200:
            data = response.json()
            # Legg til resultatene fra denne siden til den totale listen
            companies = data["_embedded"]["enheter"]
            
            # Filtrer kun aksjeselskaper (organisasjonsformkode 'AS') og ekskluder selskaper under konkursbehandling
            as_companies = [
                company for company in companies 
                if company.get('organisasjonsform', {}).get('kode') == 'AS' and company.get('status') != 'Konkurs'
            ]
            
            all_companies.extend(as_companies)
            total_fetched += len(as_companies)

            # Sjekk om vi er på siste side (siste side har ingen "_links" -> "next")
            if "next" not in data["_links"]:
                break
            else:
                page += 1  # Gå til neste side
        else:
            print(f"Feil ved forespørsel: {response.status_code}")
            break

    # Returner kun maksimalt antall ønskede resultater (f.eks. 5000)
    return all_companies[:max_results]

# Eksempel på bruk med næringskode 47.24 og by Oslo
nace_code = "47.24"
city = "Oslo"
companies = get_companies_by_nace_code_and_city(nace_code, city)

# Nå er 'companies' variabelen filtrert for kun aksjeselskaper som ikke er under konkursbehandling
print(f"Fant {len(companies)} aksjeselskaper i Oslo med næringskode {nace_code}")

Fant 58 aksjeselskaper i Oslo med næringskode 47.24


In [42]:
orgnummer = []

for i in range(len(companies)):
    orgnummer.append(companies[i]["organisasjonsnummer"])
    
print(orgnummer)

['932740265', '922388237', '916325177', '917137153', '924328770', '994277294', '916106432', '913358473', '913337441', '929006402', '912737977', '980607291', '931408577', '927069911', '914893208', '814599892', '914148375', '930107395', '925551910', '816850592', '931371290', '912753077', '930793507', '997868447', '915109004', '981313038', '983238874', '916186703', '921996543', '923239707', '976587952', '983386784', '925677418', '910856286', '916364180', '928580687', '913884256', '932006987', '832495182', '928038440', '817403352', '991368477', '917391785', '915650864', '918061932', '915255779', '918644628', '996531937', '931907409', '916100027', '979671008', '992064641', '914366518', '915647774', '914586909', '924460113', '989872109', '822674232']


In [43]:
import requests
import xml.etree.ElementTree as ET

# Funksjon for å hente regnskapsdata fra API
def get_company_financial_data(orgnr, year, regnskapstype="SELSKAP"):
    url = f"https://data.brreg.no/regnskapsregisteret/regnskap/{orgnr}"
    params = {
        'år': year,
        'regnskapstype': regnskapstype
    }
    
    headers = {'accept': 'application/xml'}
    
    # Send forespørselen til API-et
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.text  # Returner XML-data
    else:
        return f"Feil: {response.status_code}, melding: {response.text}"

# Funksjon for å trekke ut organisasjonsnummer og driftsresultat fra XML
def extract_orgnr_and_driftsresultat(xml_data):
    root = ET.fromstring(xml_data)
    
    # Hent organisasjonsnummer
    orgnr = root.find('.//virksomhet/organisasjonsnummer').text if root.find('.//virksomhet/organisasjonsnummer') is not None else "N/A"
    
    # Hent driftsresultat
    driftsresultat = root.find('.//driftsresultat/driftsresultat').text if root.find('.//driftsresultat/driftsresultat') is not None else "N/A"
    
    return orgnr, driftsresultat

# Liste over organisasjonsnumre
 # Her legger du inn organisasjonsnumrene

# År for regnskapsdata
år = 2022

# Hent og vis data for hvert organisasjonsnummer
for orgnr in orgnummer:
    xml_data = get_company_financial_data(orgnr, år)
    
    if xml_data.startswith("Feil"):
        print(xml_data)
    else:
        orgnr, driftsresultat = extract_orgnr_and_driftsresultat(xml_data)
        print(f"Organisasjonsnummer: {orgnr}, Driftsresultat: {driftsresultat}")


Feil: 404, melding: 
Organisasjonsnummer: 922388237, Driftsresultat: 173584.00
Organisasjonsnummer: 916325177, Driftsresultat: 329619.00
Organisasjonsnummer: 917137153, Driftsresultat: 638710.00
Organisasjonsnummer: 924328770, Driftsresultat: 159378.00
Organisasjonsnummer: 994277294, Driftsresultat: 126295.00
Organisasjonsnummer: 916106432, Driftsresultat: 18549346.00
Organisasjonsnummer: 913358473, Driftsresultat: -766681.00
Organisasjonsnummer: 913337441, Driftsresultat: 141727.00
Organisasjonsnummer: 929006402, Driftsresultat: 7176.00
Organisasjonsnummer: 912737977, Driftsresultat: 525195.00
Organisasjonsnummer: 980607291, Driftsresultat: -21163.00
Organisasjonsnummer: 931408577, Driftsresultat: 132846.00
Organisasjonsnummer: 927069911, Driftsresultat: 247614.00
Organisasjonsnummer: 914893208, Driftsresultat: -7651.00
Organisasjonsnummer: 814599892, Driftsresultat: 22828.00
Organisasjonsnummer: 914148375, Driftsresultat: -2089326.00
Organisasjonsnummer: 930107395, Driftsresultat: -2